In [1]:
!pip install -q transformers peft accelerate bitsandbytes yfinance
!pip install -q rouge-score
!pip install -q pynvml
# !pip install huggingface-cli
# !huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.1 MB/s eta 0:00:00


In [13]:
!pip install nsetools

In [ ]:
import os
import re
import time
import json
import random
import torch
import pandas as pd
import yfinance as yf
from peft import PeftModel
from collections import defaultdict
from datetime import date, datetime, timedelta
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer

# (Optionally set hugging face access token)
access_token = ''
#os.environ.get("HF_TOKEN")

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
SYSTEM_PROMPT = "You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. " \
    "Your answer format should be as follows:\n\n[Positive Developments]:\n1. ...\n\n[Potential Concerns]:\n1. ...\n\n[Prediction & Analysis]\nPrediction: ...\nAnalysis: ..."

# Define the base model and LoRA weights.
base_model_path = 'meta-llama/Llama-2-7b-chat-hf'
lora_model_path = 'FinGPT/fingpt-forecaster_dow30_llama2-7b_lora'


# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    token=access_token,
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,
    offload_folder="offload/",
)

# Load LoRA weights
model = PeftModel.from_pretrained(
    base_model,
    lora_model_path,
    offload_folder="offload/"
)
model = model.eval()


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    base_model_path,
     token=access_token
)
streamer = TextStreamer(tokenizer)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

adapter_model.safetensors:   0%|          | 0.00/40.0M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [1]:
import yfinance as yf

def get_company_profile(symbol):
    """Fetches company profile data using yfinance."""
    try:
        ticker = yf.Ticker(symbol)
        info = ticker.info
        history = ticker.history(period="max")
        # ipo_date = history.index.min().strftime("%Y-%m-%d")
        # Map data to the required structure
        return {
            "name": info.get("longName", "N/A"),
            "finnhubIndustry": info.get("industry", "N/A"),
            "ipo_date" : history.index.min().strftime("%Y-%m-%d"),
            "marketCapitalization": info.get("marketCap", 0) // 1_00_000,  # Convert to Lakhs for INR
            "currency": info.get("financialCurrency", "N/A"),
            "shareOutstanding": info.get("sharesOutstanding", 0) // 1_000_000,  # Convert to Millions
            "country": info.get("country", "N/A"),
            "ticker": symbol,
            "exchange": info.get("exchange", "N/A"),
        }
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None
print(get_company_profile("RELIANCE.NS"))

{'name': 'Reliance Industries Limited', 'finnhubIndustry': 'Oil & Gas Refining & Marketing', 'ipo_date': '1996-01-01', 'marketCapitalization': 169452702, 'currency': 'INR', 'shareOutstanding': 13532, 'country': 'India', 'ticker': 'RELIANCE.NS', 'exchange': 'NSI'}


In [5]:
!pip install gnews

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 16.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=46479d8f4d8fbc4ff0d77ef7b1b954b418a84a4f825dabad7eb80ee80231c13c
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [9]:
from gnews import GNews
from datetime import datetime
import time

def fetch_news_with_gnews(symbol):
    """Fetch company-specific news using the GNews library."""
    # Define company names based on symbols
    company_names = {
        "RELIANCE.NS": "Reliance Industries",
        "TCS.NS": "Tata Consultancy Services"
    }

    company_name = company_names.get(symbol, symbol)

    # Initialize GNews client
    google_news = GNews(language='en', country='IN', max_results=5)

    try:
        # Search for news
        news_articles = google_news.get_news(company_name)

        # Process the news articles
        news_list = []
        for article in news_articles:
            published_date = article.get('published date', None)

            # Parse published date if available
            if published_date:
                # Ensure we parse string dates into datetime objects
                try:
                    parsed_date = datetime.strptime(published_date, "%Y-%m-%dT%H:%M:%SZ")
                except ValueError:
                    parsed_date = datetime.now()  # Fallback to current time if parsing fails
            else:
                parsed_date = datetime.now()

            timestamp = int(time.mktime(parsed_date.timetuple()))

            news_list.append({
                "datetime": timestamp,
                "headline": article['title'],
                "summary": article['description'],
                # "url": article['url'],
            })

        return news_list

    except Exception as e:
        print(f"Error fetching news: {e}")
        return []

# Example Usage
if __name__ == "__main__":
    news = fetch_news_with_gnews("RELIANCE.NS")
    for article in news:
        print(str(article) + "\n")


{'datetime': 1736939331, 'headline': 'RIL Q3 Results: Date, Time, Earnings Schedule And Other Details - NDTV Profit', 'summary': 'RIL Q3 Results: Date, Time, Earnings Schedule And Other Details  NDTV Profit'}

{'datetime': 1736939331, 'headline': 'Reliance Industries Q3 FY25 results preview: Net profit seen up 6%, margin expansion expected - Moneycontrol', 'summary': 'Reliance Industries Q3 FY25 results preview: Net profit seen up 6%, margin expansion expected  Moneycontrol'}

{'datetime': 1736939331, 'headline': "RIL Q3 results: Will Reliance Jio, Retail, O2C perform well? Here's earnings preview - Business Today", 'summary': "RIL Q3 results: Will Reliance Jio, Retail, O2C perform well? Here's earnings preview  Business Today"}

{'datetime': 1736939331, 'headline': "Reliance Industries Q3 results 2025 date, time: Mukesh Ambani-led RIL's quarterly earnings, analyst meet schedule - ET Now", 'summary': "Reliance Industries Q3 results 2025 date, time: Mukesh Ambani-led RIL's quarterly ear

In [11]:
# # Mocking Finnhub for basic testing
# def mock_finnhub_company_profile2(symbol):
#     """Mocks Finnhub's company_profile2 API call."""
#     if symbol == "RELIANCE.NS":
#         return {
#             "name": "Reliance Industries Limited",
#             "finnhubIndustry": "Conglomerates",
#             "ipo": "1977-01-01",
#             "marketCapitalization": 1900000,
#             "currency": "INR",
#             "shareOutstanding": 6700,
#             "country": "India",
#             "ticker": "RELIANCE.NS",
#             "exchange": "NSE",
#         }
#     elif symbol == "TCS.NS":
#       return {
#             "name": "Tata Consultancy Services Ltd.",
#             "finnhubIndustry": "IT Services",
#             "ipo": "2004-08-25",
#             "marketCapitalization": 1270000,
#             "currency": "INR",
#             "shareOutstanding": 3600,
#             "country": "India",
#             "ticker": "TCS.NS",
#             "exchange": "NSE",
#         }
#     else:
#       return None

# def mock_finnhub_company_news(symbol, _from, to):
#     """Mocks Finnhub's company_news API call."""
#     if symbol == "RELIANCE.NS":
#        return [
#                 {
#                     "datetime": int(time.mktime(datetime.strptime("2023-11-01 12:00:00", "%Y-%m-%d %H:%M:%S").timetuple())),
#                     "headline": "Reliance to invest in renewable energy",
#                     "summary": "Reliance announces new investment plan",
#                 },
#                {
#                    "datetime": int(time.mktime(datetime.strptime("2023-10-30 10:00:00", "%Y-%m-%d %H:%M:%S").timetuple())),
#                    "headline": "Reliance Q3 results beat expectations",
#                    "summary": "Company beats analyst estimates",
#                }
#              ]
#     elif symbol == "TCS.NS":
#       return [
#            {
#                    "datetime": int(time.mktime(datetime.strptime("2023-11-01 12:00:00", "%Y-%m-%d %H:%M:%S").timetuple())),
#                    "headline": "TCS stock rises after strong Q3 results, special dividend announcement ",
#                    "summary": "For the quarter ended December 31, 2024, TCS reported a consolidated net profit of ₹12,444 crore, representing a 12.1 per cent increase compared to ₹11,097 crore in the same quarter last year",
#                },
#                {
#                    "datetime": int(time.mktime(datetime.strptime("2023-10-30 10:00:00", "%Y-%m-%d %H:%M:%S").timetuple())),
#                    "headline": "TCS to create 25K-seat capacity in Bengaluru with Rs 1,625 cr land deal",
#                    "summary": "Tata Consultancy Services (TCS), India's largest software exporter, is expected to increase profit margins as the BSNL agreement winds down over the next several quarters.",
#                }
#           ]
#     else:
#         return []


def mock_finnhub_company_basic_financials(symbol, type):
    """Mocks Finnhub's company_basic_financials API call"""
    if symbol == "RELIANCE.NS":
      return {
          "series":{
              "quarterly": {
                  "grossProfit": [
                          {"period":"2023-10-01", "v": 12000}
                      ],
                  "operatingProfit": [
                       {"period":"2023-10-01", "v": 8000}
                      ],
                   "netProfit":[
                       {"period":"2023-10-01", "v": 6000}
                   ]
               }
          }
      }
    elif symbol == "TCS.NS":
        return {
          "series":{
              "quarterly": {
                 "grossProfit": [
                          {"period":"2023-10-01", "v": 9000}
                      ],
                  "operatingProfit": [
                      {"period":"2023-10-01", "v": 6500}
                      ],
                   "netProfit":[
                       {"period":"2023-10-01", "v": 4800}
                    ]
                 }
          }
      }
    else:
      return { "series":{}}

def get_curday():
  return date.today().strftime("%Y-%m-%d")

def n_weeks_before(date_string, n):
  date = datetime.strptime(date_string, "%Y-%m-%d") - timedelta(days=7*n)
  return date.strftime("%Y-%m-%d")

def sample_news(news, k=5):
    return [news[i] for i in sorted(random.sample(range(len(news)), k))]

def get_company_prompt(symbol):
    profile = mock_finnhub_company_profile2(symbol)
    if not profile:
       raise Exception(f"Failed to find company profile for symbol {symbol}")
    company_template = "[Company Introduction]:\n\n{name} is a leading entity in the {finnhubIndustry} sector. Incorporated and publicly traded since {ipo}, the company has established its reputation as one of the key players in the market. As of today, {name} has a market capitalization of {marketCapitalization:.2f} in {currency}, with {shareOutstanding:.2f} shares outstanding." \
        "\n\n{name} operates primarily in the {country}, trading under the ticker {ticker} on the {exchange}. As a dominant force in the {finnhubIndustry} space, the company continues to innovate and drive progress within the industry."
    formatted_str = company_template.format(**profile)
    return formatted_str

def get_prompt_by_row(symbol, row):
  start_date = row['Start Date'] if isinstance(row['Start Date'], str) else row['Start Date'].strftime('%Y-%m-%d')
  end_date = row['End Date'] if isinstance(row['End Date'], str) else row['End Date'].strftime('%Y-%m-%d')
  term = 'increased' if row['End Price'] > row['Start Price'] else 'decreased'
  head = "From {} to {}, {}'s stock price {} from {:.2f} to {:.2f}. Company news during this period are listed below:\n\n".format(
        start_date, end_date, symbol, term, row['Start Price'], row['End Price'])

  news = json.loads(row["News"])
  news = ["[Headline]: {}\n[Summary]: {}\n".format(
        n['headline'], n['summary']) for n in news if n['date'][:8] <= end_date.replace('-', '') and \
          not n['summary'].startswith("Looking for stock market analysis and research with proves results?")]
  basics = json.loads(row['Basics'])
  if basics:
    basics = "Some recent basic financials of {}, reported at {}, are presented below:\n\n[Basic Financials]:\n\n".format(
      symbol, basics['period']) + "\n".join(f"{k}: {v}" for k, v in basics.items() if k != 'period')
  else:
      basics = "[Basic Financials]:\n\nNo basic financial reported."
  return head, news, basics

def get_current_basics(symbol, curday):
   basic = mock_finnhub_company_basic_financials(symbol, 'all')
   final_basics, basic_list, basic_dict = [], [], defaultdict(dict)
   for metric, value_list in basic['series']['quarterly'].items():
        for value in value_list:
            basic_dict[value['period']].update({metric: value['v']})
   for k, v in basic_dict.items():
      v.update({'period': k})
      basic_list.append(v)
   basic_list.sort(key=lambda x: x['period'])
   for basic in basic_list[::-1]:
       if basic['period'] <= curday:
           break
   return basic

def fetch_all_data(symbol, curday, n_weeks):
  steps = [n_weeks_before(curday, i) for i in range(n_weeks+1)][::-1]

  data = get_stock_data(symbol, steps)
  # Mock the finnhub news data fetching function
  news_list = []
  for a, row in data.iterrows():
    week_start_date = row['Start Date'].strftime('%Y-%m-%d')
    week_end_date = row['End Date'].strftime('%Y-%m-%d')
    news = mock_finnhub_company_news(symbol=symbol, _from=week_start_date, to=week_end_date)
    weekly_news = [
            {
                "date": datetime.fromtimestamp(n['datetime']).strftime('%Y%m%d%H%M%S'),
                "headline": n['headline'],
                "summary": n['summary'],
            } for n in news
        ]
    weekly_news.sort(key=lambda x: x['date'])
    news_list.append(json.dumps(weekly_news))
  data['News'] = news_list

  return data

def get_all_prompts_online(symbol, data, curday, with_basics=True):
  company_prompt = get_company_prompt(symbol)
  prev_rows = []
  for row_idx, row in data.iterrows():
    head, news, _ = get_prompt_by_row(symbol, row)
    prev_rows.append((head, news, None))
  prompt = ""
  for i in range(-len(prev_rows), 0):
    prompt += "\n" + prev_rows[i][0]
    sampled_news = sample_news(
        prev_rows[i][1],
        min(5, len(prev_rows[i][1]))
    )
    if sampled_news:
        prompt += "\n".join(sampled_news)
    else:
        prompt += "No relative news reported."
  period = "{} to {}".format(curday, n_weeks_before(curday, -1))
  if with_basics:
    basics = get_current_basics(symbol, curday)
    basics = "Some recent basic financials of {}, reported at {}, are presented below:\n\n[Basic Financials]:\n\n".format(
            symbol, basics['period']) + "\n".join(f"{k}: {v}" for k, v in basics.items() if k != 'period')
  else:
        basics = "[Basic Financials]:\n\nNo basic financial reported."
  info = company_prompt + '\n' + prompt + '\n' + basics
  prompt = info + f"\n\nBased on all the information before {curday}, let's first analyze the positive developments and potential concerns for {symbol}. Come up with 2-4 most important factors respectively and keep them concise. Most factors should be inferred from company related news. " \
        f"Then make your prediction of the {symbol} stock price movement for next week ({period}). Provide a summary analysis to support your prediction."
  return info, prompt

In [ ]:
indian_tickers = ["RELIANCE.NS", "TCS.NS"]
# Mock Stock Data
def get_stock_data(stock_symbol, steps):
  """Mocks getting stock data using yfinance."""
  data = pd.DataFrame()
  if stock_symbol == "RELIANCE.NS":
        data = pd.DataFrame({
            "Start Date": [datetime.strptime(steps[0], "%Y-%m-%d"), datetime.strptime(steps[1], "%Y-%m-%d")],
            "End Date": [datetime.strptime(steps[1], "%Y-%m-%d"), datetime.strptime(steps[-1], "%Y-%m-%d")],
            "Start Price": [2500.00, 2520.00],
            "End Price": [2520.00, 2580.00],
        })
  elif stock_symbol == "TCS.NS":
       data = pd.DataFrame({
            "Start Date": [datetime.strptime(steps[0], "%Y-%m-%d"), datetime.strptime(steps[1], "%Y-%m-%d")],
            "End Date": [datetime.strptime(steps[1], "%Y-%m-%d"), datetime.strptime(steps[-1], "%Y-%m-%d")],
            "Start Price": [3500.00, 3550.00],
            "End Price": [3550.00, 3500.00],
       })

  return data

In [ ]:

def construct_prompt(ticker, curday, n_weeks, use_basics):
  try:
      steps = [n_weeks_before(curday, n) for n in range(n_weeks + 1)][::-1]
  except Exception as e:
      raise Exception(f"Invalid date {curday}!") from e
  data = fetch_all_data(ticker, curday, n_weeks)
  data['Basics'] = [json.dumps({})] * len(data)  # to prevent error in the following get_all_prompts
  info, prompt = get_all_prompts_online(ticker, data, curday, use_basics)
  prompt = B_INST + B_SYS + SYSTEM_PROMPT + E_SYS + prompt + E_INST
  return info, prompt


def predict(ticker, date, n_weeks, use_basics):
    info, prompt = construct_prompt(ticker, date, n_weeks, use_basics)
    inputs = tokenizer(
        prompt, return_tensors='pt', padding=False
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True, streamer=streamer
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)

    torch.cuda.empty_cache()
    return info, answer

In [ ]:
# Testing with sample data
test_ticker = "TCS.NS"  # replace with other ticker in the list
test_date = get_curday()
test_n_weeks = 3
test_use_basics = True


info, answer = predict(test_ticker, test_date, test_n_weeks, test_use_basics)

print("Information:\n")
print(info)
print("-----")
print("\nResponse:\n")
print(answer)

<s> [INST]<<SYS>>
You are a seasoned stock market analyst. Your task is to list the positive developments and potential concerns for companies based on relevant news and basic financials from the past weeks, then provide an analysis and prediction for the companies' stock price movement for the upcoming week. Your answer format should be as follows:

[Positive Developments]:
1. ...

[Potential Concerns]:
1. ...

[Prediction & Analysis]
Prediction: ...
Analysis: ...
<</SYS>>

[Company Introduction]:

Tata Consultancy Services Ltd. is a leading entity in the IT Services sector. Incorporated and publicly traded since 2004-08-25, the company has established its reputation as one of the key players in the market. As of today, Tata Consultancy Services Ltd. has a market capitalization of 1270000.00 in INR, with 3600.00 shares outstanding.

Tata Consultancy Services Ltd. operates primarily in the India, trading under the ticker TCS.NS on the NSE. As a dominant force in the IT Services space, 